In [ ]:
from z3 import *
import random

# Definir as funções dos LFSRs conforme a cifra A5/1
def step_lfsr1(lfsr):
    feedback = (lfsr >> 18) ^ (lfsr >> 17) ^ (lfsr >> 16) ^ (lfsr >> 13)
    return LShR(lfsr, 1) | (feedback << 18)

def step_lfsr2(lfsr):
    feedback = (lfsr >> 21) ^ (lfsr >> 20)
    return LShR(lfsr, 1) | (feedback << 21)

def step_lfsr3(lfsr):
    feedback = (lfsr >> 22) ^ (lfsr >> 21) ^ (lfsr >> 20) ^ (lfsr >> 7)
    return LShR(lfsr, 1) | (feedback << 22)

def majority_bit(x, y, z):
    return If((Extract(0, 0, x) + Extract(0, 0, y) + Extract(0, 0, z)) >= 2, 1, 0)

# Função para simular a saída do gerador de chaves A5/1
def simulate_a5_1(lfsr1_initial, lfsr2_initial, lfsr3_initial, steps):
    lfsr1 = lfsr1_initial
    lfsr2 = lfsr2_initial
    lfsr3 = lfsr3_initial
    output_stream = []

    for _ in range(steps):
        majority = majority_bit(lfsr1, lfsr2, lfsr3)
        output_stream.append(majority)

        # Adicionar a condição correta para a comparação
        if majority == 1:
            lfsr1 = step_lfsr1(lfsr1)
            lfsr2 = step_lfsr2(lfsr2)
            lfsr3 = step_lfsr3(lfsr3)

    return output_stream

def detect_burst_of_zeros(solver, output_stream, t):
    for i in range(len(output_stream) - t + 1):
        zero_burst = And([output_stream[i + j] == 0 for j in range(t)])
        solver.add(zero_burst)

def detect_repeated_burst(solver, output_stream, t):
    for i in range(len(output_stream) - t + 1):
        for j in range(i + 1, len(output_stream) - t + 1):
            burst1 = output_stream[i:i+t]
            burst2 = output_stream[j:j+t]
            solver.add(burst1 == burst2)

def run_a5_1_simulation(t, steps):
    # Criar um solver
    solver = Solver()

    # Gerar estados iniciais aleatórios para os LFSRs
    lfsr1_initial = BitVecVal(random.randint(0, 2**19 - 1), 19)
    lfsr2_initial = BitVecVal(random.randint(0, 2**22 - 1), 22)
    lfsr3_initial = BitVecVal(random.randint(0, 2**23 - 1), 23)

    # Gerar a sequência de saída com base nesses estados
    output_stream = simulate_a5_1(lfsr1_initial, lfsr2_initial, lfsr3_initial, steps)

    # Verificar a propriedade de burst de zeros
    detect_burst_of_zeros(solver, output_stream, t)

    # Verificar a propriedade de repetição de burst
    detect_repeated_burst(solver, output_stream, t)

    # Tentar satisfazer as restrições e verificar se a propriedade ocorre
    if solver.check() == sat:
        print("Propriedades atingíveis a partir do estado inicial aleatório!")
    else:
        print("Propriedades não atingíveis a partir do estado inicial aleatório.")

# Exemplo de execução
t = 3  # Tamanho do burst
steps = 16  # Número de passos na simulação
run_a5_1_simulation(t, steps)



AttributeError: 'BitVecNumRef' object has no attribute 'bv_width'